In [10]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import numpy as np
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import ChatPromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaLLM                  

In [11]:
MODEL = "qwen3:14b-q4_K_M" # running in ollama on my local (ryzen 5 rx 3060 12GB)
db_name = "vector_db"

In [12]:
# Load environment variables

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [13]:
# Take everything in all the sub-folders of our knowledgebase 
# those are the result of scraping line campus material (scrape_shinsen.py and scrape_manabu.py)
# since you need to be logged in to access the material I saved the cookies (just ran save_cookies.py and logged in)

folders = glob.glob("knowledge_base/*")

def add_metadata(doc, doc_type):
    doc.metadata["doc_type"] = doc_type
    return doc

text_loader_kwargs = {'encoding': 'utf-8'}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    documents.extend([add_metadata(doc, doc_type) for doc in folder_docs])


In [14]:
# 1. Split raw docs into small, overlapping chunks (~300 tokens each)
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,         # roughly 300‑350 tokens
    chunk_overlap=150,
)
chunks = splitter.split_documents(documents)   

print(f"Prepared {len(chunks)} chunks.")

# 2. Create an embedding object that will respect OpenAI limits
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",  # very fast, good quality
    chunk_size=200                   # ≤200 chunks per API call → ≤~60k tokens
)

# 3. (Re)build the vector DB
if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=db_name,
)
print(f"Vectorstore ready with {vectorstore._collection.count()} chunks.")

Prepared 1109 chunks.
Vectorstore ready with 1109 chunks.


In [16]:
from few_shots import SYSTEM_PROMPT, FEW_SHOTS # added some questions with the answer I knew, and a brief explanation

messages = [("system", SYSTEM_PROMPT)]
for q, a in FEW_SHOTS:
    messages.append(("human", q))
    messages.append(("assistant", a))

# finally the placeholders for retrieval context + new question
messages.append(
    ("human",
     "【文脈資料】\n{context}\n\n"
     "【質問】\n{question}\n\n"
     "まず簡単に思考ステップを列挙し、そのあと回答を出してください。")
)

prompt = ChatPromptTemplate.from_messages(messages)


In [18]:

# 1. LLM — keep randomness low for exam‑style accuracy
llm =  OllamaLLM(
    model=MODEL,   
    temperature=0.0,     # 0–0.3 = deterministic & focused
)

# 2. Retriever — wraps your Chroma vector store
retriever = vectorstore.as_retriever(search_kwargs={"k": 6}) # Good balance: enough context to answer; still well under token limits

# 3. Put it together
qa_chain = ConversationalRetrievalChain.from_llm(
    llm       = llm,
    retriever = retriever,
    combine_docs_chain_kwargs={"prompt": prompt},
)


In [21]:
# Just testing a basic question
question = "LINE公式アカウントのUIDとは何ですか？"

response = qa_chain(               # or qa_chain.invoke(...) – same thing
    {
        "question": question,
        "chat_history": [],       
    }
)

print(response["answer"])


この質問には答えが明確なものではないため、質問を分析してみましょう。

1.  **LINE公式アカウントのUIDとは何ですか？** この質問は、LINE公式アカウントのユーザーID（uid）について尋ねているようです。ただし、この質問には明確な答えが得られません。
2.  **LINE公式アカウントの UIDを知っていても、そのUIDが無効な場合はメッセージを送信できません。** この文は、ユーザーIDが有効かどうかを確認する方法について説明しています。しかし、この質問には明確な答えが得られません。
3.  **プロフィール情報を取得するエンドポイントを使用して、ユーザーIDが有効かを確認できます。** この文は、プロフィール情報を取得するエンドポイントを使用して、ユーザーIDが有効かどうかを確認する方法について説明しています。しかし、この質問には明確な答えが得られません。

この質問には明確な答えが得られないため、回答はありません。ただし、質問の内容は、LINE公式アカウントのユーザーID（uid）に関する情報を提供することに関連しています。


In [20]:
# UI to paste the questions in
import gradio as gr

def answer_question(user_input):
    if not user_input.strip():
        return "⚠️ 質問を入力してください"
    result = qa_chain.invoke({
        "question": user_input,
        "chat_history": []
    })
    return result["answer"]

with gr.Blocks() as demo:
    gr.Markdown("## LINE公式アカウント Advanced 試験アシスタント")
    inp = gr.Textbox(label="問題文と選択肢を貼り付けてください",
                     placeholder="問題文と4つの選択肢をここに入力", lines=8)
    out = gr.Textbox(label="回答", lines=6, interactive=False)
    btn = gr.Button("解答する")
    btn.click(answer_question, inputs=inp, outputs=out)\
       .then(lambda : "", None, inp)   # clear box after answer
    demo.queue()
    
if __name__ == "__main__":
    demo.launch(server_name="0.0.0.0", server_port=7860)


* Running on local URL:  http://0.0.0.0:7860
* To create a public link, set `share=True` in `launch()`.
